In [ ]:
#Check Device for GPU
'''
Si Pytorch trouve 
GPU disponible, les données et le modèle seront poussés vers le GPU pour l'entrainement. 
'''
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Imports nécéssaires

In [ ]:
from tensorflow.python.eager.monitoring import MonitoredTimer
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from time import time
from torch import nn,optim
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import tensorflow as tf
from sklearn.metrics import accuracy_score
import numpy as np
from torch.utils.data import Dataset, DataLoader
import re
import torch
import torchvision
import math

#Information Utiles
''' acide aminé train 18105 et target 111

    acide aminé test 3520 et target 17

'''


#Maximum length est de  498

' acide aminé train 18105 et target 111\n\nacide aminé test 3520 et target 17\n\n'

Dans cette cellule :
**********
***lirefile*** vas lire le fichier en brut et renvoyé une list of list des proteine chaque proteine dans une list, et pour labels  également une list of list des targets.
*******
je définit les dictionnaires avec chaque acide et targets son numéro (ac_am_dict , targ_dict )
*******
***TransformAcideAminTRAIN*** :
retourne un dataframe avec les target et acide aminé tous de meme taille et avec des numéro entier pour facilité leur transformation en one hot
Par exemple : [3, 11, 9, 8, 9, 5, 4, 9, 17, 8, 13, 12, 3, 3,...] 

In [ ]:
#Data2
def lirefile(file_name):
    data=[]
    target=[]
    
    
    with open(file_name, 'r', encoding='utf-8') as raw_file:
        for i,line in enumerate(raw_file):
            line_content = re.sub('[\n\t]', '', line)
            if line_content not in ["end",'<>']:
                d=line_content.split(" ")
                data[-1].append(d[0])
                target[-1].append(d[1])
            elif line_content in ['<>']:
                data.append([])
                target.append([])
    return data , target

dataTrain , labelTrain =lirefile("/content/drive/MyDrive/AvignonM1_ApplicationIA/DATA/protein-secondary-structure (1).train")
dataTest , labelTest =lirefile("/content/drive/MyDrive/AvignonM1_ApplicationIA/DATA/protein-secondary-structure.test")
print(dataTest)
print(labelTest)

ac_am_dict = {
    
    "A": 0,
    "C": 1,
    "D": 2,
    "E": 3,
    "F": 4,
    "G": 5,
    "H": 6,
    "I": 7,
    "K": 8,
    "L": 9,
    "M": 10,
    "N": 11,
    "P": 12,
    "Q": 13,
    "R": 14,
    "S": 15,
    "T": 16,
    "V": 17,
    "W": 18,
    "Y": 19,
    "0": 20,
    
    
     
    
}

targ_dict = {
    
    "_": 0,
    "e": 1,
    "h": 2,
    "0": 3,
    
    
    
}
 

 
def TransformAcideAmin(DataFrame):
     #Normalisation pour avoir des protein de meme taill 
     for i,row in DataFrame.iterrows():
        if 498 != len(row['acideAmine']):
            more= 498 - len(row['acideAmine'])
            row['acideAmine'].extend(['0']*more)
        #remplissage avec padding des 0
        if 498 != len(row['label']):
            more=498 - len(row['label'])
            row['label'].extend(['0']*more)
        #une fois le row transformé je l'ajout au data frame
        DataFrame.at[i,'acideAmine'] = row['acideAmine']
        DataFrame.at[i,'label'] = row['label']

    #a cette étapes j'ai un dataframes de proteines et target avec le padding mais encore avec les données brute  
     for i,row in DataFrame.iterrows():
        a = []
        #chaq acide  sera remplacé par son numéro du dictionnaire
        for acide in row['acideAmine']:    
            a.append(ac_am_dict[acide])
            


        DataFrame.at[i,"acideAmine"] = a
      
        target = []
        #chaq target  sera remplacé par son numéro du dictionnaire
        for trgt in row['label']:
            target.append(targ_dict[trgt])
            
        DataFrame.at[i,"label"] = target
        

     return DataFrame



            




#Je crée le dataframe qui sera l'entrée de TransformAcideAmin
train = pd.DataFrame({'acideAmine':dataTrain,'label':labelTrain})
test = pd.DataFrame({'acideAmine':dataTest,'label':labelTest})


FinaleTRAIN = TransformAcideAmin(train)
FinaleTest = TransformAcideAmin(test)
print(FinaleTest)
#ici à la sortie j'aurai des dataframe avec les données de meme taille et leurs numéros adéquat

#je transforme mes données en one hot
x_train = np.array(FinaleTRAIN['acideAmine'].to_list())
y_train = np.array(FinaleTRAIN['label'].to_list())
x_test = np.array(FinaleTest['acideAmine'].to_list())
y_test = np.array(FinaleTest['label'].to_list())


#transformé en torch
x_train = torch.from_numpy(tf.one_hot(x_train, depth =21).numpy()).to(device)
x_test = torch.from_numpy(tf.one_hot(x_test, depth =21).numpy()).to(device)
y_train = torch.from_numpy(tf.one_hot(y_train, depth =4).numpy()).to(device)
y_test = torch.from_numpy(tf.one_hot(y_test, depth =4).numpy()).to(device)
#par exemple : [0., 0., 0.,  ..., 0., 0., 0.]

print(x_train.shape)
print(y_train.shape)
print(y_test)
print(x_test)


#cette fonction pour cree un dataset je l'utilise pas
class Dataset(Dataset):

    def __init__(self,X,Y):
        
        self.x_data = X
       
            
        self.y_data =Y
        
        
    # we can call len(dataset) to return the size
    def __len__(self):
        return len(self.y_data)


        
    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        #
        return self.x_data[index],self.y_data[index]



from torch.utils.data import DataLoader, TensorDataset


# Je Cree TensorDataSet avec la fonction prédef
traindata = TensorDataset(x_train,y_train)
testdata = TensorDataset(x_test,y_test)
#tensor([[[0., 0., 0.,  ..., 0., 0., 0.]
#J'appele le dataloader pour un lot de données de 16 pour les deux train et test
loadtrain = DataLoader(traindata, batch_size =16 )
loadtest = DataLoader(testdata, batch_size = 16)

[['E', 'N', 'L', 'K', 'L', 'G', 'F', 'L', 'V', 'K', 'Q', 'P', 'E', 'E', 'P', 'W', 'F', 'Q', 'T', 'E', 'W', 'K', 'F', 'A', 'D', 'K', 'A', 'G', 'K', 'D', 'L', 'G', 'F', 'E', 'V', 'I', 'K', 'I', 'A', 'V', 'P', 'D', 'G', 'E', 'K', 'T', 'L', 'N', 'A', 'I', 'D', 'S', 'L', 'A', 'A', 'S', 'G', 'A', 'K', 'G', 'F', 'V', 'I', 'C', 'T', 'P', 'D', 'P', 'K', 'L', 'G', 'S', 'A', 'I', 'V', 'A', 'K', 'A', 'R', 'G', 'Y', 'D', 'M', 'K', 'V', 'I', 'A', 'V', 'D', 'D', 'Q', 'F', 'V', 'N', 'A', 'K', 'G', 'K', 'P', 'M', 'D', 'T', 'V', 'P', 'L', 'V', 'M', 'M', 'A', 'A', 'T', 'K', 'I', 'G', 'E', 'R', 'Q', 'G', 'Q', 'E', 'L', 'Y', 'K', 'E', 'M', 'Q', 'K', 'R', 'G', 'W', 'D', 'V', 'K', 'E', 'S', 'A', 'V', 'M', 'A', 'I', 'T', 'A', 'N', 'E', 'L', 'D', 'T', 'A', 'R', 'R', 'R', 'T', 'T', 'G', 'S', 'M', 'D', 'A', 'L', 'K', 'A', 'A', 'G', 'F', 'P', 'E', 'K', 'Q', 'I', 'Y', 'Q', 'V', 'P', 'T', 'K', 'S', 'N', 'D', 'I', 'P', 'G', 'A', 'F', 'D', 'A', 'A', 'N', 'S', 'M', 'L', 'V', 'Q', 'H', 'P', 'E', 'V', 'K', 'H', 'W', 'L'

**Model **
Le choix du model est un Gated Recurrent Unit (GRU) bidirectionnelle car il a donnée de meilleur résultat que le unidirectionnelle



In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


# Hyper-parameters 
num_epochs = 2
batch_size = 64
learning_rate = 0.1
input_size = 21
hidden_size = 16
num_layers = 2


class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):

        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.num_layers = 2
        self.hidden_size = 16
        self.gru = nn.GRU(input_size, 16, num_layers*2, batch_first=True,bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, 4)
        self.relu = nn.ReLU()
        
        
    def forward(self, x,h0):
          
          out, o = self.gru(x, h0)  
          
          a = nn.Sequential(
            self.relu,
            self.fc)
          
          out = a(out)
          # out: (n, 10)
          return out , o
          
    def init_hidden(self, batch_size):
        
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        


model =  NeuralNetwork(input_size, hidden_size, num_layers)
model.to(device)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (gru): GRU(21, 16, num_layers=4, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=32, out_features=4, bias=True)
  (relu): ReLU()
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
#Cellule que j'utilise pas 
dataset = Dataset(x_train,y_train)
train_loader = DataLoader(dataset=dataset,batch_size=64, shuffle=True)
datasettest = Dataset(x_test,y_test)
test_loader = DataLoader(dataset=datasettest,batch_size=64, shuffle=True)

Train

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader.dataset):
        
        h = model.init_hidden(len(X))
        p , h =model(X,h)
        loss = loss_fn(p, y)
        

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  ")

Test

j'ai créer la fonction test qui run le modèle et vérifier l'accuracy qui est tout simplement un compteur  qui compte chaque prédiction correspondant à l'étiquette.

In [ ]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    print(num_batches)
    test_loss, correct = 0, 0
    size_no_padding=0

    with torch.no_grad():
        for X, y in dataloader:
            
            h = model.init_hidden(len(X))
            pred , h =model(X,h)


            
            #Etape pour enlevé le padding qui faussé les résulat et me donner des accuracy 
            #illogiquement élevé
            mask = (torch.ones(y.shape[0], y.shape[1])*(y.shape[2]-1)).to(device) != y.argmax(dim=2)
            pred_no_padding = torch.masked_select(pred.argmax(dim=2), mask)
            y_no_padding = torch.masked_select(y.argmax(dim=2), mask)

            correct += (pred_no_padding == y_no_padding).type(torch.float).sum().item()   # compte du nombre de prédiction correctes
            size_no_padding += len(pred_no_padding)
            
            loss = loss_fn(pred, y)
            test_loss += loss_fn(pred, y).item()

    #''' test_loss /= num_batches
    correct /= size_no_padding 
    
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
loss_fn = nn.MSELoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=0.1,momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.001)


for t in range(20):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(loadtrain, model, loss_fn, optimizer)
    test_loop(loadtest, model,  loss_fn)
    
print("Done!")

Epoch 1
-------------------------------
loss: 0.252125  
loss: 0.046541  
2
Test Error: 
 Accuracy: 54.3%, Avg loss: 0.078148 

Epoch 2
-------------------------------
loss: 0.072395  
loss: 0.045737  
2
Test Error: 
 Accuracy: 54.5%, Avg loss: 0.076390 

Epoch 3
-------------------------------
loss: 0.070839  
loss: 0.042950  
2
Test Error: 
 Accuracy: 54.8%, Avg loss: 0.072635 

Epoch 4
-------------------------------
loss: 0.060507  
loss: 0.040405  
2
Test Error: 
 Accuracy: 56.0%, Avg loss: 0.071448 

Epoch 5
-------------------------------
loss: 0.054146  
loss: 0.038544  
2
Test Error: 
 Accuracy: 58.2%, Avg loss: 0.068757 

Epoch 6
-------------------------------
loss: 0.054742  
loss: 0.036766  
2
Test Error: 
 Accuracy: 59.8%, Avg loss: 0.066876 

Epoch 7
-------------------------------
loss: 0.059151  
loss: 0.035820  
2
Test Error: 
 Accuracy: 62.1%, Avg loss: 0.064601 

Epoch 8
-------------------------------
loss: 0.062716  
loss: 0.034327  
2
Test Error: 
 Accuracy: 62.6